In [1]:
import pandas as pd
import numpy as np
import requests
import os

In [2]:
os.environ['IP2LOCATION_TOKEN'] = 'token'

In [3]:
# # Replace with your actual token and database code
# token = os.getenv('IP2LOCATION_TOKEN')
# database_code = 'PX11LITECSV'
# dataname = 'px11'
# # Construct the URL
# url = f"https://www.ip2location.com/download/?token={token}&file={database_code}"

# # Set the destination path for the downloaded file
# output_path = f"./{dataname}.zip"

# try:
#     # Send GET request to download the file
#     response = requests.get(url, stream=True)
#     response.raise_for_status()  # Raise an error for bad responses

#     # Write the file in chunks to handle large downloads
#     with open(output_path, 'wb') as file:
#         for chunk in response.iter_content(chunk_size=1024):
#             if chunk:
#                 file.write(chunk)

#     print(f"Download successful! File saved to {output_path}")

# except requests.exceptions.RequestException as e:
#     print(f"Download failed: {e}")


Download successful! File saved to ./px11.zip


## Loading Data and Finding IP Addresses

In [4]:
df = pd.read_csv("./IP2PROXY-LITE-PX11.CSV", header=None)

In [5]:
def ip_number_to_address(ip_number):
    return f"{(ip_number >> 24) & 255}.{(ip_number >> 16) & 255}.{(ip_number >> 8) & 255}.{ip_number & 255}"

rename_dict = {
    0: 'ip_num_low',
    1: 'ip_num_high',
    2: 'proxytype',
    3: 'country_code',
    4: 'country_name',
    5: 'region',
    6: 'city',
    7: 'isp',
    8: 'domain',
    9: 'usage_type',
    10: 'asn',
    11: 'company',
    12: 'threat_level',
    13: 'threat_type',
    14: 'provider'
}

df.rename(columns=rename_dict, inplace=True)

# Apply to DataFrame
df['ip_low'] = df['ip_num_low'].apply(ip_number_to_address)
df['ip_high'] = df['ip_num_high'].apply(ip_number_to_address)

## Exploring IP Ranges

In [6]:
df['ip_difference'] = df['ip_num_high'] - df['ip_num_low']
# Filter rows where the difference is exactly 1
diff1 = df[(df['ip_difference'] == 1) | (df['ip_difference'] == -1)].drop(columns=['ip_difference'])
diffplus = df[df['ip_difference'] > 1].drop(columns=['ip_difference'])

In [13]:
diffplus.head()

,ip_num_low,ip_num_high,proxytype,country_code,country_name,region,city,isp,domain,usage_type,asn,company,threat_level,threat_type,provider,ip_low,ip_high
2386,37321406,37321410,PUB,RO,Romania,Bucuresti,Bucharest,Techoff SRV Limited,apiversa.com,DCH,47890,Unmanaged Ltd,4,SPAM/SCANNER,-,2.57.122.190,2.57.122.194
2479,37373997,37373999,PUB,GB,United Kingdom of Great Britain and Northern I...,England,London,Stumpner Netservice UG (haftungsbeschraenkt),stumpnernet.de,DCH,62240,Clouvider Limited,30,-,SurfShark VPN,2.58.72.45,2.58.72.47
3838,50792960,50793471,PUB,IN,India,Maharashtra,Mumbai,Amazon Data Services India,amazon.com,DCH,16509,Amazon.com Inc.,1,-,Amazon API Gateway,3.7.10.0,3.7.11.255
4963,50945536,50945791,PUB,GB,United Kingdom of Great Britain and Northern I...,England,London,Amazon Data Services UK,amazon.com,DCH,16509,Amazon.com Inc.,1,-,Amazon API Gateway,3.9.94.0,3.9.94.255
5111,50991360,50991487,PUB,GB,United Kingdom of Great Britain and Northern I...,England,London,Amazon Data Services UK,amazon.com,DCH,16509,Amazon.com Inc.,1,-,Amazon API Gateway,3.10.17.0,3.10.17.127


## Ensuring we only have one column with no meaningful data

In [12]:
for col in list(diffplus.columns):
    if len(df[col].unique()) < 2:
        print(col)

proxytype


In [15]:
df = df.drop("proxytype", axis=1)

In [20]:
df.columns

Index(['ip_num_low', 'ip_num_high', 'country_code', 'country_name', 'region',
       'city', 'isp', 'domain', 'usage_type', 'asn', 'company', 'threat_level',
       'threat_type', 'provider', 'ip_low', 'ip_high', 'ip_difference'],
      dtype='object')

## Save as Parquet

In [21]:
df.to_parquet("/var/tmp/outfiles/cleaned_ips.parquet")

In [22]:
df.head()

,ip_num_low,ip_num_high,country_code,country_name,region,city,isp,domain,usage_type,asn,company,threat_level,threat_type,provider,ip_low,ip_high,ip_difference
0,16782178,16782178,JP,Japan,Tokyo,Tokyo,I2TS Inc.,i2ts.com,DCH,-,-,30,-,-,1.0.19.98,1.0.19.98,0
1,16782320,16782320,JP,Japan,Tokyo,Tokyo,I2TS Inc.,i2ts.com,DCH,-,-,30,-,-,1.0.19.240,1.0.19.240,0
2,16782350,16782350,JP,Japan,Tokyo,Tokyo,I2TS Inc.,i2ts.com,DCH,-,-,30,-,-,1.0.20.14,1.0.20.14,0
3,16783214,16783214,JP,Japan,Tokyo,Tokyo,I2TS Inc.,i2ts.com,DCH,-,-,30,-,-,1.0.23.110,1.0.23.110,0
4,16783399,16783399,JP,Japan,Tokyo,Tokyo,I2TS Inc.,i2ts.com,DCH,-,-,30,-,-,1.0.24.39,1.0.24.39,0
